In [31]:
!pip install torchmetrics
import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as data
import math
import copy
import torch
import torch.nn as nn
import torch.nn.functional as F
import random
import pandas as pd
import numpy as np
from   sklearn.preprocessing   import LabelEncoder          # кодировка категорий
import pandas as pd
from torch.autograd import Variable
import numpy as np
import torch.functional as F
from   sklearn.preprocessing   import LabelEncoder          # кодировка категорий
from   sklearn.decomposition import PCA
from   sklearn import linear_model
import matplotlib.pyplot as plt                   # постороение графиков
import datetime
from sklearn.preprocessing import StandardScaler
from sklearn.manifold import TSNE
import torch.optim as optim
import torch
from torch.utils.data import DataLoader, TensorDataset
import torchmetrics

In [32]:
# Читаем файл с компетенциями
df = pd.read_excel('/content/Dataset_L_4_25_diff_5_COLAB.xlsx')

In [33]:
df['SKILLS'] = df['SKILLS'].astype(str)

# Длинна каждого посчитаем длинну
df['list_of_skill'] = df['SKILLS'].apply(lambda x: x.split(','))

In [34]:
df['cnt_skill'] = df['list_of_skill'].apply(lambda x: len(x))

In [35]:
df.head(5)

,SKILLS,list_of_skill,cnt_skill
0,"ЯП,ПЛАТФОРМЫ,ФРЕЙМВОРКИ JAVA 8 11,SPRING BOOT,...","[ЯП, ПЛАТФОРМЫ, ФРЕЙМВОРКИ JAVA 8 11, SPRING B...",7
1,"ЯНДЕКС МЕТРИКА,УПРАВЛЕНИЕ ПРОЕКТАМИ,ЯНДЕКС ДИР...","[ЯНДЕКС МЕТРИКА, УПРАВЛЕНИЕ ПРОЕКТАМИ, ЯНДЕКС ...",7
2,"ЯНДЕКС МЕТРИКА,КОНКУРЕНТНАЯ АНАЛИТИКА,СОЗДАНИЕ...","[ЯНДЕКС МЕТРИКА, КОНКУРЕНТНАЯ АНАЛИТИКА, СОЗДА...",7
3,"ЯНДЕКС МЕТРИКА,ИНТЕРНЕТ РЕКЛАМА,МЕДИАПЛАНИРОВА...","[ЯНДЕКС МЕТРИКА, ИНТЕРНЕТ РЕКЛАМА, МЕДИАПЛАНИР...",7
4,"ЯНДЕКС МЕТРИКА,GOOGLE ANALYTICS,SQL,КОНТЕКСТНА...","[ЯНДЕКС МЕТРИКА, GOOGLE ANALYTICS, SQL, КОНТЕК...",7


In [36]:
df_skill = df[(df.cnt_skill > 4) & (df.cnt_skill < 26)]

In [37]:
data = df_skill

In [38]:
len(data)

84571

In [39]:
# Создаем словарь
vocabulary = []
vocabulary.append('[PAD]')
vocabulary.append('[MASK]')
for i, row  in data.iterrows():
    for j in data["list_of_skill"][i]:
        if j not in vocabulary:
            vocabulary.append(j)
word2idx = {w: idx for (idx, w) in enumerate(vocabulary)}
idx2word = {idx: w for (idx, w) in enumerate(vocabulary)}
vocabulary_size = len(vocabulary)

In [40]:
vocabulary_size

35787

In [41]:
from collections import Counter

# Создадим список всех слов в списке слов
all_words = [word for sublist in data.list_of_skill for word in sublist]

# Подсчитаем количество повторений каждого слова
word_counts = Counter(all_words)

# Выведем результат
print(word_counts)

Counter({'РАБОТА В КОМАНДЕ': 22926, 'ГРАМОТНАЯ РЕЧЬ': 20770, 'ПОЛЬЗОВАТЕЛЬ ПК': 15875, 'ДЕЛОВОЕ ОБЩЕНИЕ': 10994, 'ВЕДЕНИЕ ПЕРЕГОВОРОВ': 10455, 'ТЕЛЕФОННЫЕ ПЕРЕГОВОРЫ': 9453, 'АКТИВНЫЕ ПРОДАЖИ': 9064, 'ДЕЛОВАЯ ПЕРЕПИСКА': 8878, 'ОРГАНИЗАТОРСКИЕ НАВЫКИ': 8572, 'РАБОТА С БОЛЬШИМ ОБЪЕМОМ ИНФОРМАЦИИ': 7814, 'НАВЫКИ ПРОДАЖ': 7341, 'ГРАМОТНОСТЬ': 7258, 'РАБОТА В УСЛОВИЯХ МНОГОЗАДАЧНОСТИ': 7167, 'ОРИЕНТАЦИЯ НА РЕЗУЛЬТАТ': 6889, 'УМЕНИЕ РАБОТАТЬ В КОМАНДЕ': 6664, 'ДЕЛОВАЯ КОММУНИКАЦИЯ': 6549, 'ПОИСК И ПРИВЛЕЧЕНИЕ КЛИЕНТОВ': 6252, 'АНГЛИЙСКИЙ ЯЗЫК': 6058, 'ЗАКЛЮЧЕНИЕ ДОГОВОРОВ': 5789, 'УПРАВЛЕНИЕ ПЕРСОНАЛОМ': 5075, 'ОТВЕТСТВЕННОСТЬ': 4877, 'КЛИЕНТООРИЕНТИРОВАННОСТЬ': 4745, 'КОММУНИКАБЕЛЬНОСТЬ': 4661, 'ОБУЧЕНИЕ И РАЗВИТИЕ': 4447, 'МНОГОЗАДАЧНОСТЬ': 4447, '1С ПРЕДПРИЯТИЕ 8': 4385, 'ПРЯМЫЕ ПРОДАЖИ': 4153, 'УПРАВЛЕНИЕ ПРОЕКТАМИ': 4099, 'B2B ПРОДАЖИ': 3953, 'РАЗВИТИЕ ПРОДАЖ': 3933, 'КРЕАТИВНОСТЬ': 3628, 'ОБУЧЕНИЕ ПЕРСОНАЛА': 3617, 'УМЕНИЕ РАБОТАТЬ В УСЛОВИЯХ МНОГОЗАДАЧНОСТИ': 3430, 'CRM': 3400, 'ПРОВ

In [42]:
from collections import Counter

# Создадим список всех слов в списке слов
all_words = [word for sublist in data.list_of_skill for word in sublist]

# Подсчитаем количество повторений каждого слова
word_counts = Counter(all_words)

# Получим список слов, которые встречаются всего один раз
unique_words = [word for word, count in word_counts.items() if count == 1]

# Выведем результат
print(unique_words)

['ЯП', 'ПЛАТФОРМЫ', 'ФРЕЙМВОРКИ JAVA 8 11', 'JUNIT ХРАНИЛИЩА', 'ОЧЕРЕДИ APAC', 'СУДЫ', 'ТУРБИННОЕ ОБОРУДОВАНИЕ', 'КОНТРОЛЬ ДЗ', 'МОНИТОРИНГ ДЗ', 'ЭНЕРГОМЕНЕДЖМЕНТ', 'АССИСТИРОВАНИЕ В СТОМАТОЛОГИИ', 'ЗНАНИЕ ОСНОВ МОНТАЖА ЭЛЕКТРОПРОВОДКИ', 'ПРОЕКТИРОВАНИЕ СИСТЕМ ВИДЕОНАБЛЮДЕНИЯ', 'ПРОЕКТИРОВАНИЕ КОНТРОЛЯ ДОСТУПА', 'ОХРАННО ПОЖАРНОЙ СИГНАЛИЗАЦИИ', 'ПЕРИМЕТРАЛЬНОЙ СИГНАЛИЗАЦИИ', 'ЭКСПРЕССИЯ БЕЛКОВ', 'HPLC', 'BRADFORD', 'NI NTA', 'ГЕЛЬ ФИЛЬТРАЦИЯ', 'ЦЕНООБРАЗОВАНИЕ В СТРОИТЕЛЬСТВЕ', 'АНАЛИЗ СМЕТ', 'ОПРЕДЕЛЕНИЕ НАЧАЛЬНОЙ МИНИМАЛЬНОЙ ЦЕНЫ', 'СЕБЕСТОИМОСТЬ В СТРОИТЕЛЬСТВЕ', 'ОТЧЕТНАЯ КАЛЬКУЛЯЦИЯ ПО ФАКТИЧЕСКОЙ СЕБЕСТОИМОСТИ', 'ЮРИДИЧЕСКИЕ РИСКИ', 'ОТХОД', 'ГРАФИК ОТПУСКОВ', 'ОФОРМЛЕНИЕ СОТРУДНИКОВ', 'ШЛИФОВКА АВТОМОБИЛЕЙ', 'ПОДГОТОВКА АВТОМОБИЛЕЙ', 'ОРГАНИЗАЦИЯ РАБОТЫ КОЛЛЕКТИВА', 'СБОРОЧНЫЕ ТЕХНОЛОГИИ', 'КОНТРОЛЬ ОБЕСПЕЧЕНИЯ УЧАСТКА КОМПЛЕКТУЮЩИМИ', 'ФРЕЗЕРОВАНИЕ', 'НАПИСАНИЕ ПРОГРАММ', 'КИМ', 'ЗНАНИЕ ПРОЦЕССОВ И ТЕХНОЛОГИЙ КРОВЕЛЬНЫХ И ГИДРОИЗОЛЯЦИОННЫХ РАБОТ', 'КАТЕГОРИЯ В', 'ОМНИКАНАЛЬНАЯ 

In [43]:
len(unique_words)

24280

In [44]:
## Преобразуйте объект Counter в DataFrame
word_count_df = pd.DataFrame(list(word_counts.items()), columns=['Skill', 'Count'])

In [45]:
word_count_df.to_excel('word_count_df.xlsx')

In [46]:
# Случайная сортировка списков
for index, row in data.iterrows():
    random.shuffle(row['list_of_skill'])

In [47]:
len_tokens = 25

In [48]:
data['list_of_skill'] = data['list_of_skill'].apply(lambda x: x + ['[PAD]' for _ in range(len_tokens - len(x))])

In [49]:
data['target']= data['list_of_skill'].apply(lambda x: x[0])

In [50]:
data['seq'] = data['list_of_skill'].apply(lambda x: ['[MASK]'] + x[1:] if len(x) > 0 else [])

In [51]:
data

,SKILLS,list_of_skill,cnt_skill,target,seq
0,"ЯП,ПЛАТФОРМЫ,ФРЕЙМВОРКИ JAVA 8 11,SPRING BOOT,...","[ОЧЕРЕДИ APAC, ЯП, SPRING BOOT, JUNIT ХРАНИЛИЩ...",7,ОЧЕРЕДИ APAC,"[[MASK], ЯП, SPRING BOOT, JUNIT ХРАНИЛИЩА, ФРЕ..."
1,"ЯНДЕКС МЕТРИКА,УПРАВЛЕНИЕ ПРОЕКТАМИ,ЯНДЕКС ДИР...","[УПРАВЛЕНИЕ КОМАНДОЙ, АНАЛИТИЧЕСКОЕ МЫШЛЕНИЕ, ...",7,УПРАВЛЕНИЕ КОМАНДОЙ,"[[MASK], АНАЛИТИЧЕСКОЕ МЫШЛЕНИЕ, INTERNET MARK..."
2,"ЯНДЕКС МЕТРИКА,КОНКУРЕНТНАЯ АНАЛИТИКА,СОЗДАНИЕ...",[СОЗДАНИЕ ПРЕЗЕНТАЦИОННЫХ МАТЕРИАЛОВ О КОМПАНИ...,7,СОЗДАНИЕ ПРЕЗЕНТАЦИОННЫХ МАТЕРИАЛОВ О КОМПАНИИ,"[[MASK], GOOGLE ТАБЛИЦЫ, ЯНДЕКС МЕТРИКА, КОНКУ..."
3,"ЯНДЕКС МЕТРИКА,ИНТЕРНЕТ РЕКЛАМА,МЕДИАПЛАНИРОВА...","[ИНТЕРНЕТ РЕКЛАМА, ТАРГЕТИНГ, МЕДИАПЛАНИРОВАНИ...",7,ИНТЕРНЕТ РЕКЛАМА,"[[MASK], ТАРГЕТИНГ, МЕДИАПЛАНИРОВАНИЕ, АНАЛИЗ ..."
4,"ЯНДЕКС МЕТРИКА,GOOGLE ANALYTICS,SQL,КОНТЕКСТНА...","[B2B МАРКЕТИНГ, ВНУТРЕННИЙ КОНТРОЛЬ, ЯНДЕКС МЕ...",7,B2B МАРКЕТИНГ,"[[MASK], ВНУТРЕННИЙ КОНТРОЛЬ, ЯНДЕКС МЕТРИКА, ..."
...,...,...,...,...,...
84566,"ЭЛЕКТРОИНСТРУМЕНТ,СВЕРЛЕНИЕ,РАСПИЛОВКА,СБОРКА ...","[СВЕРЛЕНИЕ, МОНТАЖНЫЕ РАБОТЫ, ЭЛЕКТРОИНСТРУМЕН...",6,СВЕРЛЕНИЕ,"[[MASK], МОНТАЖНЫЕ РАБОТЫ, ЭЛЕКТРОИНСТРУМЕНТ, ..."
84567,"ЭЛЕКТРОМОНТЕР,МОНТАЖНЫЕ РАБОТЫ,ОБСЛУЖИВАНИЕ ЗД...","[3 ГРУППА ПО ЭЛЕКТРОБЕЗОПАСНОСТИ, МОНТАЖНЫЕ РА...",6,3 ГРУППА ПО ЭЛЕКТРОБЕЗОПАСНОСТИ,"[[MASK], МОНТАЖНЫЕ РАБОТЫ, ЭЛЕКТРОМОНТЕР, ЧТЕН..."
84568,"ЮРИДИЧЕСКОЕ СОПРОВОЖДЕНИЕ,ВЫСОКАЯ ОТВЕТСТВЕННО...","[ЮРИДИЧЕСКОЕ СОПРОВОЖДЕНИЕ, НАВЫКИ ВЕДЕНИЯ ПРЕ...",6,ЮРИДИЧЕСКОЕ СОПРОВОЖДЕНИЕ,"[[MASK], НАВЫКИ ВЕДЕНИЯ ПРЕТЕНЗИОННО ИСКОВОЙ Р..."
84569,"ЮРИДИЧЕСКОЕ СОПРОВОЖДЕНИЕ,ОФОРМЛЕНИЕ ДОКУМЕНТА...","[ПРЕТЕНЗИОННАЯ РАБОТА, ГОСУДАРСТВЕННЫЕ ЗАКУПКИ...",6,ПРЕТЕНЗИОННАЯ РАБОТА,"[[MASK], ГОСУДАРСТВЕННЫЕ ЗАКУПКИ, РАБОТА С ГОС..."


In [52]:
#Сортировка полей skills_id в датасете
#data['skills_id_sort'] = data['skills_id'].apply(sorted)

In [53]:
#data.to_excel('data_sort.xlsx')

In [54]:
def tokenize_dataset(data):
    # Создание нового столбца со всеми токенизированными элементами
    data['target_id'] = data['target'].apply(lambda x:  word2idx[x])
    data['skills_id'] = data['seq'].apply(lambda x: [word2idx[word] for word in x])
    return data

In [55]:
data = tokenize_dataset(data)

In [56]:
#data.to_excel('data.xlsx')

In [57]:
data

,SKILLS,list_of_skill,cnt_skill,target,seq,target_id,skills_id
0,"ЯП,ПЛАТФОРМЫ,ФРЕЙМВОРКИ JAVA 8 11,SPRING BOOT,...","[ОЧЕРЕДИ APAC, ЯП, SPRING BOOT, JUNIT ХРАНИЛИЩ...",7,ОЧЕРЕДИ APAC,"[[MASK], ЯП, SPRING BOOT, JUNIT ХРАНИЛИЩА, ФРЕ...",8,"[1, 2, 5, 7, 4, 3, 6, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1,"ЯНДЕКС МЕТРИКА,УПРАВЛЕНИЕ ПРОЕКТАМИ,ЯНДЕКС ДИР...","[УПРАВЛЕНИЕ КОМАНДОЙ, АНАЛИТИЧЕСКОЕ МЫШЛЕНИЕ, ...",7,УПРАВЛЕНИЕ КОМАНДОЙ,"[[MASK], АНАЛИТИЧЕСКОЕ МЫШЛЕНИЕ, INTERNET MARK...",12,"[1, 13, 14, 15, 11, 9, 10, 0, 0, 0, 0, 0, 0, 0..."
2,"ЯНДЕКС МЕТРИКА,КОНКУРЕНТНАЯ АНАЛИТИКА,СОЗДАНИЕ...",[СОЗДАНИЕ ПРЕЗЕНТАЦИОННЫХ МАТЕРИАЛОВ О КОМПАНИ...,7,СОЗДАНИЕ ПРЕЗЕНТАЦИОННЫХ МАТЕРИАЛОВ О КОМПАНИИ,"[[MASK], GOOGLE ТАБЛИЦЫ, ЯНДЕКС МЕТРИКА, КОНКУ...",17,"[1, 18, 9, 16, 13, 19, 20, 0, 0, 0, 0, 0, 0, 0..."
3,"ЯНДЕКС МЕТРИКА,ИНТЕРНЕТ РЕКЛАМА,МЕДИАПЛАНИРОВА...","[ИНТЕРНЕТ РЕКЛАМА, ТАРГЕТИНГ, МЕДИАПЛАНИРОВАНИ...",7,ИНТЕРНЕТ РЕКЛАМА,"[[MASK], ТАРГЕТИНГ, МЕДИАПЛАНИРОВАНИЕ, АНАЛИЗ ...",21,"[1, 23, 22, 26, 9, 24, 25, 0, 0, 0, 0, 0, 0, 0..."
4,"ЯНДЕКС МЕТРИКА,GOOGLE ANALYTICS,SQL,КОНТЕКСТНА...","[B2B МАРКЕТИНГ, ВНУТРЕННИЙ КОНТРОЛЬ, ЯНДЕКС МЕ...",7,B2B МАРКЕТИНГ,"[[MASK], ВНУТРЕННИЙ КОНТРОЛЬ, ЯНДЕКС МЕТРИКА, ...",32,"[1, 30, 9, 27, 28, 31, 29, 0, 0, 0, 0, 0, 0, 0..."
...,...,...,...,...,...,...,...
84566,"ЭЛЕКТРОИНСТРУМЕНТ,СВЕРЛЕНИЕ,РАСПИЛОВКА,СБОРКА ...","[СВЕРЛЕНИЕ, МОНТАЖНЫЕ РАБОТЫ, ЭЛЕКТРОИНСТРУМЕН...",6,СВЕРЛЕНИЕ,"[[MASK], МОНТАЖНЫЕ РАБОТЫ, ЭЛЕКТРОИНСТРУМЕНТ, ...",35762,"[1, 9987, 8406, 8575, 35780, 5131, 0, 0, 0, 0,..."
84567,"ЭЛЕКТРОМОНТЕР,МОНТАЖНЫЕ РАБОТЫ,ОБСЛУЖИВАНИЕ ЗД...","[3 ГРУППА ПО ЭЛЕКТРОБЕЗОПАСНОСТИ, МОНТАЖНЫЕ РА...",6,3 ГРУППА ПО ЭЛЕКТРОБЕЗОПАСНОСТИ,"[[MASK], МОНТАЖНЫЕ РАБОТЫ, ЭЛЕКТРОМОНТЕР, ЧТЕН...",1728,"[1, 9987, 9296, 173, 1198, 35781, 0, 0, 0, 0, ..."
84568,"ЮРИДИЧЕСКОЕ СОПРОВОЖДЕНИЕ,ВЫСОКАЯ ОТВЕТСТВЕННО...","[ЮРИДИЧЕСКОЕ СОПРОВОЖДЕНИЕ, НАВЫКИ ВЕДЕНИЯ ПРЕ...",6,ЮРИДИЧЕСКОЕ СОПРОВОЖДЕНИЕ,"[[MASK], НАВЫКИ ВЕДЕНИЯ ПРЕТЕНЗИОННО ИСКОВОЙ Р...",47,"[1, 11189, 8553, 299, 4313, 1007, 0, 0, 0, 0, ..."
84569,"ЮРИДИЧЕСКОЕ СОПРОВОЖДЕНИЕ,ОФОРМЛЕНИЕ ДОКУМЕНТА...","[ПРЕТЕНЗИОННАЯ РАБОТА, ГОСУДАРСТВЕННЫЕ ЗАКУПКИ...",6,ПРЕТЕНЗИОННАЯ РАБОТА,"[[MASK], ГОСУДАРСТВЕННЫЕ ЗАКУПКИ, РАБОТА С ГОС...",1419,"[1, 134, 35783, 338, 35782, 47, 0, 0, 0, 0, 0,..."


In [58]:
data.reset_index(drop=True, inplace=True)

In [59]:
#data.drop_duplicates(subset='skills_id_sort', inplace=True)

In [60]:
# Assuming data is your pandas DataFrame
dataset = TensorDataset(torch.tensor(data['skills_id']), torch.tensor(data['target_id']))

In [61]:
from torch.utils.data import TensorDataset, random_split


# 95% - train, на 5% Валидируемся.
train_size = int(0.95 * len(dataset))
val_size = len(dataset) - train_size

# Разобьем на training и validation случайным образом.
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

print('{:>5,} training samples'.format(train_size))
print('{:>5,} validation samples'.format(val_size))

80,342 training samples
4,229 validation samples


In [62]:
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

# Зададим размер батча для DataLoaders
batch_size = 512

# Создадим DataLoaders для training and validation.
# Примеры для train берем случайным образом.
train_dataloader = DataLoader(
            train_dataset,  # Обучающие примеры.
            sampler = RandomSampler(train_dataset), # Выбор батча случайно.
            batch_size = batch_size # Обучаем с заданным размером батча.
        )

# Для проверки порядок значения не имеет поэтому можно прочитать последовательно.
validation_dataloader = DataLoader(
            val_dataset, # Валидационные примеры.
            sampler = SequentialSampler(val_dataset), # Подаем батчи последовательно.
            batch_size = batch_size # Валидируем с заданным размером батча.
        )

In [63]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [64]:
class MultiHeadAttention(nn.Module):
    def __init__(self, d_model, num_heads):
        super(MultiHeadAttention, self).__init__()
        assert d_model % num_heads == 0, "d_model must be divisible by num_heads"

        self.d_model = d_model
        self.num_heads = num_heads
        self.d_k = d_model // num_heads

        self.W_q = nn.Linear(d_model, d_model, bias=False)
        self.W_k = nn.Linear(d_model, d_model, bias=False)
        self.W_v = nn.Linear(d_model, d_model, bias=False)
        self.W_o = nn.Linear(d_model, d_model, bias=False)

    def scaled_dot_product_attention(self, Q, K, V, mask ):
        attn_scores = torch.matmul(Q, K.transpose(-2, -1)) / math.sqrt(self.d_k)
        attn_scores = attn_scores.masked_fill(mask == 0, -1e9)
        attn_probs = torch.softmax(attn_scores, dim=-1)
        #Зануление весов "снизу" там где софтмакс равномерен
        attn_probs = attn_probs.masked_fill(mask.transpose(-2, -1)==False, 0.0)

        output = torch.matmul(attn_probs, V)
        return output

    def split_heads(self, x):
        batch_size, seq_length, d_model = x.size()
        return x.view(batch_size, seq_length, self.num_heads, self.d_k).transpose(1, 2)

    def combine_heads(self, x):
        batch_size, _, seq_length, d_k = x.size()
        return x.transpose(1, 2).contiguous().view(batch_size, seq_length, self.d_model)

    def forward(self, Q, K, V, mask ):
        Q = self.split_heads(self.W_q(Q.to(device)))
        K = self.split_heads(self.W_k(K.to(device)))
        V = self.split_heads(self.W_v(V.to(device)))

        attn_output = self.scaled_dot_product_attention(Q, K, V, mask.to(device))
        output = self.W_o(self.combine_heads(attn_output))
        return output

In [65]:
class EncoderLayer(nn.Module):
    def __init__(self, d_model, num_heads, d_ff, dropout):
        super(EncoderLayer, self).__init__()
        self.norm1 = nn.BatchNorm1d(len_tokens, affine=False)
        self.norm2 = nn.BatchNorm1d(len_tokens, affine=False)
        self.dropout = nn.Dropout(dropout)
        self.multihead_attention = MultiHeadAttention(d_model, num_heads)
        self.feed_forward = nn.Sequential(
            nn.Linear(d_model, d_ff),
            nn.ReLU(),
            nn.Linear(d_ff, d_model),
            nn.BatchNorm1d(len_tokens)
        )

    def forward(self, src_embedded, src_mask):
        src_embedded_normalized = self.norm1(src_embedded.to(device))
        src_attention_output = self.multihead_attention(src_embedded_normalized, src_embedded_normalized, src_embedded_normalized, src_mask)

        src_attention_output = src_attention_output + src_embedded
        src_attention_output_normalized = self.norm2(src_attention_output)

        src_ff_output = self.feed_forward(src_attention_output_normalized)
        src_ff_output = self.dropout(src_ff_output)

        src_output = src_attention_output + src_ff_output
        return src_output

In [66]:
class Transformer_Encoder(nn.Module):
    def __init__(self, d_model, num_heads, num_layers, d_ff, dropout):
        super(Transformer_Encoder, self).__init__()
        self.embedding = nn.Embedding(vocabulary_size, d_model, padding_idx = 0)

        self.encoder_layers = nn.ModuleList([EncoderLayer(d_model, num_heads, d_ff, dropout) for _ in range(num_layers)])

        self.fc = nn.Linear(d_model, vocabulary_size, bias=False)
        self.dropout = nn.Dropout(dropout)
        self.Softmax = nn.Softmax(dim=-1)

    def generate_mask(self, src):
        src_mask = (src != 0).unsqueeze(1).unsqueeze(2)
        return src_mask

    def forward(self, seq):
        src_mask = self.generate_mask(seq)
        src_embedded = self.dropout(self.embedding(seq))

        for enc_layer in self.encoder_layers:
            src_embedded = enc_layer(src_embedded, src_mask)

        output = self.fc(src_embedded)
        return output

In [67]:
d_model = 24 ### 16, 24, 32 ...
num_heads = 8
num_layers = 3 ### Попробовать 2, 3, 12, 18, 24
d_ff = d_model * 5
dropout = 0.1
Transformer_Encoder = Transformer_Encoder(d_model, num_heads, num_layers, d_ff, dropout)

In [68]:
Transformer_Encoder.to(device)

Transformer_Encoder(
  (embedding): Embedding(35787, 24, padding_idx=0)
  (encoder_layers): ModuleList(
    (0-2): 3 x EncoderLayer(
      (norm1): BatchNorm1d(25, eps=1e-05, momentum=0.1, affine=False, track_running_stats=True)
      (norm2): BatchNorm1d(25, eps=1e-05, momentum=0.1, affine=False, track_running_stats=True)
      (dropout): Dropout(p=0.1, inplace=False)
      (multihead_attention): MultiHeadAttention(
        (W_q): Linear(in_features=24, out_features=24, bias=False)
        (W_k): Linear(in_features=24, out_features=24, bias=False)
        (W_v): Linear(in_features=24, out_features=24, bias=False)
        (W_o): Linear(in_features=24, out_features=24, bias=False)
      )
      (feed_forward): Sequential(
        (0): Linear(in_features=24, out_features=120, bias=True)
        (1): ReLU()
        (2): Linear(in_features=120, out_features=24, bias=True)
        (3): BatchNorm1d(25, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
    )
  )
  (fc): 

In [69]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(Transformer_Encoder.parameters(), lr=0.01, betas=(0.9, 0.98), eps=1e-9)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, patience=5, factor=0.9, verbose=True)
accuracy = torchmetrics.Accuracy(task="multiclass", num_classes = vocabulary_size, ignore_index=0).to(device)

Transformer_Encoder.train()
epoch_loss_avg = []
for epoch in range(500):
    losses = []  # список для сохранения всех loss внутри эпохи
    Accuracys = []  # список для сохранения всех loss внутри эпохи
    total_correct = 0  # переменная для сохранения общего числа правильных предсказаний
    total_samples = 0  # переменная для сохранения общего числа обработанных примеров
    for src_data, tgt_data in train_dataloader:
        src_data = src_data.to(device)
        tgt_data = tgt_data.to(device)
        optimizer.zero_grad()
        output = Transformer_Encoder(src_data)
        output_skill = output[:, 0, :]
        loss = criterion(output_skill, tgt_data)
        acc = accuracy(output_skill, tgt_data)
     #   acc = accuracy(output.contiguous().view(-1, vocabulary_size).argmax(dim=-1), tgt_data.contiguous().view(-1))
        losses.append(loss.item())  # добавляем значение loss в список
        Accuracys.append(acc.item())
        loss.backward()
        optimizer.step()

    epoch_loss = sum(losses) / len(losses)  # среднее значение loss для эпохи
    epoch_loss_avg.append(epoch_loss)
    scheduler.step(epoch_loss)  # Вставить scheduler.step() здесь
    epoch_Accuracy = sum(Accuracys) / len(Accuracys)  # среднее значение loss для эпохи
    print(f"Epoch: {epoch + 1}, Epoch_loss: {epoch_loss:.4f}, Accuracy: {epoch_Accuracy:.4f}" )

/usr/local/lib/python3.10/dist-packages/torch/optim/lr_scheduler.py:28: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn("The verbose parameter is deprecated. Please use get_last_lr() "


Epoch: 1, Epoch_loss: 7.6474, Accuracy: 0.0220
Epoch: 2, Epoch_loss: 6.8066, Accuracy: 0.0301
Epoch: 3, Epoch_loss: 6.3988, Accuracy: 0.0362
Epoch: 4, Epoch_loss: 6.1152, Accuracy: 0.0410
Epoch: 5, Epoch_loss: 5.8809, Accuracy: 0.0466
Epoch: 6, Epoch_loss: 5.6607, Accuracy: 0.0533
Epoch: 7, Epoch_loss: 5.4740, Accuracy: 0.0613
Epoch: 8, Epoch_loss: 5.3044, Accuracy: 0.0706
Epoch: 9, Epoch_loss: 5.1472, Accuracy: 0.0800
Epoch: 10, Epoch_loss: 5.0204, Accuracy: 0.0916
Epoch: 11, Epoch_loss: 4.8979, Accuracy: 0.1043
Epoch: 12, Epoch_loss: 4.7930, Accuracy: 0.1141
Epoch: 13, Epoch_loss: 4.7062, Accuracy: 0.1250
Epoch: 14, Epoch_loss: 4.6255, Accuracy: 0.1343
Epoch: 15, Epoch_loss: 4.5472, Accuracy: 0.1430
Epoch: 16, Epoch_loss: 4.4788, Accuracy: 0.1537
Epoch: 17, Epoch_loss: 4.4153, Accuracy: 0.1596
Epoch: 18, Epoch_loss: 4.3621, Accuracy: 0.1666
Epoch: 19, Epoch_loss: 4.3123, Accuracy: 0.1743
Epoch: 20, Epoch_loss: 4.2593, Accuracy: 0.1806
Epoch: 21, Epoch_loss: 4.2166, Accuracy: 0.1878
E

KeyboardInterrupt: 

In [ ]:
plt.plot(epoch_loss_avg)

In [ ]:
input_data = []
for i in range(vocabulary_size):
    input_data.append(i)

In [ ]:
input_data = torch.tensor(input_data)

In [ ]:
import torch
import torch.nn as nn

# Assume you have a model called `my_model` with an embedding layer
embedding_layer = Transformer_Encoder.embedding  # Get the embedding layer

# Create a sample input data
#input_data = torch.tensor([1, 2, 3, 4, 5])  # example input data

# Pass the input data through the embedding layer
embeddings = embedding_layer(input_data.to(device))

# Now you have the embeddings for the input data
print(embeddings)

In [ ]:
!pip install hdbscan

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE
import hdbscan


embeddings = embeddings.cpu()
# Загрузка эмбеддингов слов
#embeddings = embeddings_np
embeddings_np = embeddings.detach().numpy()

# Сокращение размерности с помощью t-SNE
tsne = TSNE(n_components=3, perplexity=3)
embeddings_tsne = tsne.fit_transform(embeddings_np)



# Кластеризация с помощью HDBSCAN
clusterer = hdbscan.HDBSCAN(min_cluster_size=6)
clusters = clusterer.fit_predict(embeddings_tsne)

# Визуализация результатов
plt.scatter(embeddings_tsne[:, 0], embeddings_tsne[:, 1], c=clusters)
plt.colorbar()
plt.show()

In [ ]:
# Получение меток классов
labels = clusterer.labels_

# Создание словаря для сопоставления слов и меток классов
word_clusters = dict(zip(vocabulary, labels))

In [ ]:
word_clusters

In [ ]:
# Создание DataFrame из словаря
df = pd.DataFrame(list(word_clusters.items()), columns=['Слово', 'Метка'])

# Запись DataFrame в Excel
df.to_excel('/content/word_clusters.xlsx', index=False)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE
import hdbscan

embeddings = embeddings.cpu()
# Преобразование тензора embeddings в массив numpy
embeddings_np = embeddings.detach().numpy()

# Сокращение размерности с помощью t-SNE
tsne = TSNE(n_components=2, perplexity=50, learning_rate=200)
embeddings_tsne = tsne.fit_transform(embeddings_np)

# Кластеризация с помощью HDBSCAN
clusterer = hdbscan.HDBSCAN(min_cluster_size=7)
clusters = clusterer.fit_predict(embeddings_tsne)

# Поиск индексов элементов, относящихся к кластеру -1
indices_to_keep = np.where(clusters != -1)[0]

# Фильтрация embeddings_tsne, оставляем только элементы, не относящиеся к кластеру -1
filtered_embeddings_tsne = embeddings_tsne[indices_to_keep]

# Визуализация результатов
plt.figure(figsize=(10, 10))
plt.scatter(filtered_embeddings_tsne[:, 0], filtered_embeddings_tsne[:, 1], c=clusters[indices_to_keep], cmap='rainbow', s=10, alpha=0.8)
plt.colorbar()
plt.show()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE
import hdbscan

embeddings = embeddings.cpu()
# Преобразование тензора embeddings в массив numpy
embeddings_np = embeddings.detach().numpy()

# Сокращение размерности с помощью t-SNE
tsne = TSNE(n_components=2, perplexity=50, learning_rate=200)
embeddings_tsne = tsne.fit_transform(embeddings_np)

min_cluster_sizes = range(7, 22)  # Значения min_cluster_size

# Цикл для отображения карт с разными значениями min_cluster_size
for min_cluster_size in min_cluster_sizes:
    # Кластеризация с помощью HDBSCAN
    clusterer = hdbscan.HDBSCAN(min_cluster_size=min_cluster_size)
    clusters = clusterer.fit_predict(embeddings_tsne)

    # Поиск индексов элементов, относящихся к кластеру -1
    indices_to_keep = np.where(clusters != -1)[0]

    # Фильтрация embeddings_tsne, оставляем только элементы, не относящиеся к кластеру -1
    filtered_embeddings_tsne = embeddings_tsne[indices_to_keep]

    # Визуализация результатов
    plt.figure(figsize=(10, 10))
    plt.scatter(filtered_embeddings_tsne[:, 0], filtered_embeddings_tsne[:, 1], c=clusters[indices_to_keep], cmap='rainbow', s=10, alpha=0.8)
    plt.colorbar()
    plt.title(f'min_cluster_size={min_cluster_size}')  # Заголовок с текущим значением min_cluster_size
    plt.show()